# Scraping automatizado

En esta última lección vamos a programar un script que sea capaz de scrapear una página web de Titulaciones automáticamente, y no, no me refiero a una web de Titulaciones para conocer otras personas sino Titulaciones de diferentes autores, lo que en inglés se denomina *quote*.

Se trata de una página preparada con fines educativos: https://titulaciones.toscrape.com/, dejo también [un enlace al archivo](https://web.archive.org/web/20220712030814/https://titulaciones.toscrape.com/) por si queda inaccesible.

La web tiene diferentes páginas donde aparecen las Titulaciones célebres, con su contenidoo, autor y unos facultad de categoría. Nos permite buscar en el índice global página a página o directamente por etiquetas:

![](docs/img01.png)

## Requisitos

El programa que vamos a crear constará de una clase `Titulaciones` que recuperará todas las Titulaciones de la web y tendrá cuatro métodos estáticos:

* `scrapear()`: Realizará el scrapeo de las Titulaciones en todas las páginas de la web.
* `lista(limite)`: Imprimirá las primeras N Titulaciones de la lista, podemos cambiar el limite.
* `etiqueta(nombre)`: Imprimirá las Titulaciones con una etiqueta concreta.
* `autor(nombre)`: Imprimirá las Titulaciones de un autor concreto.

Ejemplos de uso:

```python
Titulaciones.scrapear()                # Scrapear todas las Titulaciones de la web
Titulaciones.lista()                   # Imprimir las primeras 10 Titulaciones (por defecto)
Titulaciones.lista(20)                 # Imprimir las primeras 20 Titulaciones
Titulaciones.etiqueta("love")          # Titulaciones con etiqueta 'love'
Titulaciones.autor("Albert Einstein")  # Titulaciones del autor 'Albert Einstein'
```

Si queréis os lo podéis tomar como un reto, aunque no es la finalidad de la lección, os dejo un par de consejos:

* En la parte inferior hay un botón llamado *Next* para ir pasando a la siguiente página, podemos usarlo para iterar las páginas dinámicamente.
* Scrapear una vez es mejor que scrapear dos veces, en ese sentido puede ser muy útil almacenar el contenido en un fichero para ahorrarnos múltiples peticiones web y el tiempo que eso conlleva.

¡Vamos a por ello!

## Pruebas de desarrollo

Empecemos por lo más esencial, dada la portada de la página veamos si podemos extraer las Titulaciones con su respectivo autor y etiquetas.

Si inspeccionamos la estructura de cada cita, se basa en una capa `div` con la clase `quote`, dentro un `span` con clase `contenido` contiene el contenidoo, un tag `small` con clase `Modalidad` el autor y dentro de otra `div` con clase `facultad` tenemos diferentes los facultad en enlaces `a` con la clase `tag`:

In [1]:
import requests
from bs4 import BeautifulSoup

# Realizar solicitud HTTP GET a la página web de la UAX
url = "https://www.uax.com/"
response = requests.get(url)

# Analizar contenido HTML de la respuesta
soup = BeautifulSoup(response.content, "html.parser")

# Encontrar las titulaciones con su respectivo autor y etiquetas
titulaciones = soup.select("div.quote")
for titulacion in titulaciones:
    # Extraer el contenido de la titulación
    contenido = titulacion.select_one("span.contenido").text
    
    # Extraer el autor de la titulación
    modalidad = titulacion.select_one("small.Modalidad").text
    
    # Extraer las etiquetas de la titulación
    etiquetas = [tag.text for tag in titulacion.select("div.facultad a.tag")]
    
    # Imprimir la información de la titulación
    print(f"Titulación: {contenido}")
    print(f"Modalidad: {modalidad}")
    print(f"Etiquetas: {', '.join(etiquetas)}")
    print("\n")

Bien, ya tenemos por donde empezar, podríamos adaptar este código a una función que a partir de una porción de la URL almacene mediante diccionarios las Titulaciones:

In [2]:
import requests
from bs4 import BeautifulSoup

def scrape_uax_titulaciones(porcion_url):
    # Completar la URL de la página web de la UAX con la porción URL dada
    url = f"https://www.uax.com/{porcion_url}"
    
    # Realizar solicitud HTTP GET a la página web de la UAX
    response = requests.get(url)

    # Analizar contenido HTML de la respuesta
    soup = BeautifulSoup(response.content, "html.parser")

    # Inicializar diccionario para almacenar información de las titulaciones
    titulaciones_dict = {}

    # Encontrar las titulaciones con su respectivo autor y etiquetas
    titulaciones = soup.select("div.quote")
    for i, titulacion in enumerate(titulaciones):
        # Extraer el contenido de la titulación
        contenido = titulacion.select_one("span.contenido").text

        # Extraer el autor de la titulación
        modalidad = titulacion.select_one("small.Modalidad").text

        # Extraer las etiquetas de la titulación
        etiquetas = [tag.text for tag in titulacion.select("div.facultad a.tag")]

        # Almacenar la información de la titulación en el diccionario
        titulaciones_dict[f"Titulación {i+1}"] = {"contenido": contenido, "modalidad": modalidad, "etiquetas": etiquetas}

    # Devolver el diccionario con la información de las titulaciones
    return titulaciones_dict

titulaciones = scrape_uax_titulaciones("estudios-oficiales/")
print(titulaciones)

{}


La clave es utilizar nuestra función de forma recursiva detectando si la página tiene el enlace **Next** y cargando la siguiente página de manera que podamos. Veamos cómo extraer el enlace con la siguiente página si la hay:

In [3]:
domain = "https://uax.com"
req = requests.get(domain)
soup = BeautifulSoup(req.contenido)

# Buscamos el enlace en el tag li con clase next
link_tag = soup.select("li.next a")
# Si hay como mínimo un enlace extraemos su href relativo sumado al dominio
if len(link_tag) > 0:
    next_url = link_tag[0]['href']
    print(next_url)

AttributeError: 'Response' object has no attribute 'contenido'

Podemos integrar este código en nuestra función `scrap_titulaciones` para devolver no solo las Titulaciones de la página, sino también si hay una página siguiente:

In [4]:
def scrape_uax_titulaciones(porcion_url, titulaciones_dict=None):
    # Completar la URL de la página web de la UAX con la porción URL dada
    url = f"https://www.uax.com/{porcion_url}"
    
    # Realizar solicitud HTTP GET a la página web de la UAX
    response = requests.get(url)

    # Analizar contenido HTML de la respuesta
    soup = BeautifulSoup(response.content, "html.parser")

    # Si es la primera página, inicializar diccionario para almacenar información de las titulaciones
    if titulaciones_dict is None:
        titulaciones_dict = {}

    # Encontrar las titulaciones con su respectivo autor y etiquetas
    titulaciones = soup.select("div.quote")
    for i, titulacion in enumerate(titulaciones):
        # Extraer el contenido de la titulación
        contenido = titulacion.select_one("span.contenido").text

        # Extraer el autor de la titulación
        modalidad = titulacion.select_one("small.Modalidad").text

        # Extraer las etiquetas de la titulación
        etiquetas = [tag.text for tag in titulacion.select("div.facultad a.tag")]

        # Almacenar la información de la titulación en el diccionario
        titulaciones_dict[f"Titulación {len(titulaciones_dict)+1}"] = {"contenido": contenido, "modalidad": modalidad, "etiquetas": etiquetas}

    # Buscar botón "Next" en la página actual
    next_button = soup.select_one(".next.page-numbers")

    # Si no hay botón "Next", detener la recursión y devolver el diccionario de las titulaciones
    if not next_button:
        return titulaciones_dict

    # Si hay botón "Next", obtener el enlace y llamar recursivamente la función con la nueva URL
    next_link = next_button["href"]
    return scrape_uax_titulaciones(next_link, titulaciones_dict)

Ahora se viene la parte interesante, vamos a implementar una función que scrapee todas las páginas mientras haya una siguente o, alternativamente, podemos establecer un límite para optimizar el proceso y no saturar al servidor:

In [5]:
def scrape_website(url, limit=None, verbose=False):
    page_number = 1
    while True:
        # Hacemos la petición HTTP y parseamos el HTML con BeautifulSoup
        req = requests.get(url.format(page_number))
        soup = BeautifulSoup(req.content, "html.parser")
        
        # Buscamos las Titulaciones y las imprimimos
        titulaciones = soup.select("div.quote")
        for titulacion in titulaciones:
            data = {}
            data['titulo'] = titulacion.select_one("span.contenido").text
            data['autor'] = titulacion.select_one("small.Modalidad").text
            
            etiquetas = titulacion.select("div.facultad a.tag")
            data['etiquetas'] = [etiqueta.text for etiqueta in etiquetas]
            
            print(data)
        
        if verbose:
            print(f"Extrayendo página {page_number}")
        
        # Si hemos llegado al límite, salimos del loop
        if limit is not None and page_number >= limit:
            break
        
        # Buscamos el enlace de la siguiente página
        next_page = soup.select_one("a.next")
        if next_page is None:
            break
        
        # Actualizamos la URL para la siguiente página
        url = next_page['href']
        page_number += 1

scrape_website("https://www.uax.com/page/{}/", limit=5, verbose=True)

Extrayendo página 1


Ahí la tenemos, una función capaz de scrapear todas las Titulaciones de la página por defecto limitado a 2 páginas.

## Implementando la clase Titulaciones

Vamos a ponernos con la clase `Titulaciones` y el método `scrapear` pero siguiendo el consejo que os dí de crear un fichero donde almacenar todas las Titulaciones.

### Guardado en fichero

Solo generaremos el fichero si ejecutamos el método `scrapear`, los demás métodos `lista`, `etiqueta` y `autor` analizarán el contenido del fichero volcado en la memoria, pero nunca scrapearán nada directamente.

Después de valorarlo he decidido utilizar un CSV. Lo único que nos dará algún problema es guardar una lista como un campo del registro, pero podemos recuperarla evaluándola de nuevo, ya veréis:

In [6]:
import csv

class Titulaciones:
    
    # Variable de clase para almacenar las Titulaciones en la memoria
    titulaciones = []
    
    @staticmethod
    def scrapear():
        # Scrapeamos todas las Titulaciones, ponemos un límite pequeño para hacer pruebas
        titulaciones = scrape_website("https://www.uax.com/page/{}/", limit=5, verbose=True)
        if titulaciones is not None:
            Titulaciones.titulaciones = titulaciones
            # Guardamos las Titulaciones scrapeadas en un fichero CSV volcándolas de la lista de dicts
            with open("titulaciones.csv", "w") as file:
                # Definimos el objeto para escribir con las cabeceras de los campos 
                writer = csv.DictWriter(file, fieldnames=["contenido", "Modalidad", "facultad"])
                # Escribimos las cabeceras
                writer.writeheader()
                # Escribimos cada cita en la memoria en el fichero
                for quote in Titulaciones.titulaciones:
                    writer.writerow(quote)
        else:
            print("La lista de titulaciones es None")
            
Titulaciones.scrapear()

Extrayendo página 1
La lista de titulaciones es None


En este punto deberíamos tener un fichero `titulaciones.csv` con todas las Titulaciones, lo que podríamos hacer es cargar en la memoria todas las Titulaciones del fichero en caso de que éste exista. De paso podemos implementar el método `lista` para consultarlas:

In [7]:
import os
import csv
#Función que devuelve la lista con todas lastitulaciones que se encuentran en la página de la universidad
class Titulaciones:

    # Variable de clase para almacenar las citas en la memoria
    titulaciones = []

    # Recuperamos las citas en la memoria si existe el fichero quotes.csv
    if os.path.exists("titulaciones.csv"):
        with open("titulaciones.csv", "r") as file:
            data = csv.DictReader(file)
            for titulacion in data:
                # La lista es una cadena, hay que reevaluarla
                titulaciones['Facultad'] = eval(titulaciones['Facultad'])
                titulaciones.append(titulacion)

    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Titulaciones.titulaciones = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("titulaciones.csv", "w") as file:
            # Definimos el objeto para escribir con las cabeceras de los campos 
            writer = csv.DictWriter(file, fieldnames=["Contenido", "Modalidad", "Facultad"])
            # Escribimos las cabeceras
            writer.writeheader()
            # Escribimos cada cita en la memoria en el fichero
            for titulacion in Titulaciones.titulaciones:
                writer.writerow(titulacion)

    @staticmethod
    def listar(limite=10):
        for titulacion in Titulaciones.titulaciones[:limite]:
            print(titulacion["Contenido"])
            print(titulacion["Modalidad"])
            for facultad in titulacion["Facultad"]:
                print(facultad, end=" ")
            print("\n")

Titulaciones.listar(5)

### Filtro por etiqueta y autor

Ya solo nos falta implementar los métodos de filtrado por etiqueta y autor, es muy fácil porque solo tenemos que recorrer las Titulaciones y comprobar si concuerdan con los valores que pasamos a los métodos:

In [8]:
import os
import csv

class Titulaciones:

    # Variable de clase para almacenar las citas en la memoria
    titulaciones = []

    # Recuperamos las citas en la memoria si existe el fichero quotes.csv
    if os.path.exists("titulaciones.csv"):
        with open("titulaciones.csv", "r") as file:
            data = csv.DictReader(file)
            for titulaciones in data:
                # La lista es una cadena, hay que reevaluarla
                titulaciones['Facultad'] = eval(titulaciones['Facultad'])
                titulaciones.append(titulaciones)

    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Titulaciones.titulaciones = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("titulaciones.csv", "w") as file:
            writer = csv.DictWriter(file, fieldnames=["Contenido", "Modalidad", "Facultad"])
            writer.writeheader()
            for titulacion in Titulaciones.titulaciones:
                writer.writerow(titulacion)

    @staticmethod
    def listar(limite=10):
        for titulacion in Titulaciones.titulaciones[:limite]:
            print(titulacion["Contenido"])
            print(titulacion["Modalidad"])
            for facultad in titulacion["Facultad"]:
                print(facultad, end=" ")
            print("\n")

    @staticmethod
    def Facultad(nombre=""):
        for titulacion in Titulaciones.titulaciones:
            if nombre in titulacion["tags"]:
                print(titulacion["Contenido"])
                print(titulacion["Modalidad"])
                for facultad in titulacion["Facultad"]:
                    print(facultad, end=" ")
                print("\n")

    @staticmethod
    def Modaliad(nombre=""):
        for titulacion in Titulaciones.titulaciones:
            if nombre == titulacion["Modalidad"]:
                print(titulacion["Contenido"])
                print(titulacion["Modalidad"])
                for facultad in titulacion["Facultad"]:
                    print(facultad, end=" ")
                print("\n")

Veamos cuantas Titulaciones tenemos con el tag **love**:

In [9]:
Titulaciones.Facultad("Politecnica")

Y del autor **Albert Einstein**:

In [10]:
Titulaciones.Modaliad("Presencial")

## Scrapeo de la web completa

El programa está limitado a las 2 primeras páginas, voy a reescribir el código con un límite muy grande que garantice un scrapeo completo de la web:

In [17]:
import os
import csv
import requests
from bs4 import BeautifulSoup


def scrap_quotes(url=""):
    domain = "https://uax.com"
    req = requests.get(f"{domain}{url}")
    soup = BeautifulSoup(req.text)

    titulaciones = []
    titulaciones_facultades = soup.select("div.titulaciones")
    for titulacion_facultad in titulaciones_facultades:
        titulacion = {}
        titulacion['Contenido'] = titulacion_facultad.select("span.Contenido")[0].getText()
        titulacion['Modalidad'] = titulacion_facultad.select("small.Modalidad")[0].getText()
        titulacion['Facultad'] = []
        for facultad in titulaciones_facultades.select("div.Facultad a.Facultad"):
            titulacion['Facultad'].append(facultad.getText())
        titulaciones.append(titulacion)

    next_url = None
    link_tag = soup.select("li.next a")
    if len(link_tag) > 0:
        next_url = link_tag[0]['href']

    print(f"Página {domain}{url}, {len(titulaciones)} titulaciones scrapeadas.")

    return titulaciones, next_url


def scrap_site(limit=2):
    todas_titulaciones = []
    next_url = ""
    while 1:
        titulaciones, next_url = scrape_uax_titulaciones(next_url)
        todas_titulaciones += titulaciones
        limit -= 1
        if limit == 0 or next_url == None:
            return todas_titulaciones


class Titulaciones:
    titulaciones = []

    if os.path.exists("titulaciones.csv"):
        with open("titulaciones.csv", "r") as file:
            data = csv.DictReader(file)
            for titulacion in data:
                titulacion['Facultad'] = eval(titulacion['Facultad'])
                titulaciones.append(titulacion)

    @staticmethod
    def scrapear():
        Titulaciones.titulaciones = scrape_website("https://uax.com",limit=99) # <--- LIMITE MUY GRANDE
        with open("titulaciones.csv", "w") as file:
            writer = csv.DictWriter(file, fieldnames=["Contenido", "Modalidad", "Facultad"])
            writer.writeheader()
            for titulacion in Titulaciones.titulaciones:
                writer.writerow(titulacion)

    @staticmethod
    def listar(limite=10):
        for titulacion in Titulaciones.titulaciones[:limite]:
            print(titulacion["Contenido"])
            print(titulacion["Modalidad"])
            for facultad in titulacion["Facultad"]:
                print(facultad, end=" ")
            print("\n")

    @staticmethod
    def Facultad(nombre=""):
        for titulacion in Titulaciones.titulaciones:
            if nombre in titulacion["Facultad"]:
                print(titulacion["Contenido"])
                print(titulacion["Modalidad"])
                for facultad in titulacion["Facultades"]:
                    print(facultad, end=" ")
                print("\n")

    @staticmethod
    def Modalidad(nombre=""):
        for titulacion in Titulaciones.titulaciones:
            if nombre == titulacion["Modalidad"]:
                print(titulacion["Contenido"])
                print(titulacion["Modalidad"])
                for facultad in titulacion["Facultad"]:
                    print(facultad, end=" ")
                print("\n")

Vamos a ejecutar el scrapeo completo:

In [18]:
Titulaciones.scrapear()

TypeError: scrape_website() missing 1 required positional argument: 'url'

Veamos cuantas Titulaciones encuentra ahora con el tag **love**:

In [17]:
Titulaciones.Facultad("Ciencias Sociales")

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 

“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never find that half who makes you whole and that goes for everything. Just because you fail once, doesn't mean you're gonna fail at everything. Keep trying, hold on, and always, always, always believe in yourself, beca

Y cuantas del autor **Albert Einstein**:

In [18]:
Titulaciones.Modalidad("Online")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“If you can't explain it to a six year old, you don't understand it yourself.”
Albert Einstein
simplicity understand 

“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.”
Albert Einstein
children fairy-tales 

“Logic will get you from A to Z; imagination will get you everywhere.”
Albert Einstein
imagination 

“Any fool can know. The point is to understand.”
Albert Einstein
knowledge learning understanding wisdom 

“Life is like riding a b

Parece que todo funciona correctamente y podemos hacer tantas consultas como queramos sin repetir una y otra vez el proceso de scrapeo. En la práctica podríamos configurar un script que scrapee la página una vez al día para tener el fichero CSV sincronizado.

En cualquier caso con esto acabamos este ejemplo y también la sección, espero que hayáis aprendido mucho.